# GET información de los partidos

In [1]:
import requests
import json 
import os
import regex 

headers = {
	"X-RapidAPI-Key": "9501611b6amsh9cc47bda49e4b1ep1ac05cjsne72511c5c3d4",
	"X-RapidAPI-Host": "sofascore.p.rapidapi.com"
}

### Obtenemos id de todos los equipo de LaLiga en la Api de sofaescore

In [2]:
url = "https://sofascore.p.rapidapi.com/teams/search"

equipos = ["Real Madrid", "Barcelona", "Atlético de Madrid", "Valencia", "Sevilla", "Villarreal", "Real Sociedad", "Real Betis", "Athletic Club", "Celta Vigo", "Almeria", "Getafe", "Mallorca", "Girona", "Granada", "Cadiz", "Alavés", "Rayo Vallecano", "Osasuna", "Las Palmas"]

ids_equipos = []

for equipo in equipos:
    response = requests.get(url, headers=headers, params={"name": equipo})
    data = response.json()
    if data['teams']:
        equipo_id = data['teams'][0]['id']
        ids_equipos.append(equipo_id)
        print(f"GET ID: {equipo}: {equipo_id}")

GET ID: Real Madrid: 2829
GET ID: Barcelona: 2817
GET ID: Atlético de Madrid: 77879
GET ID: Valencia: 2828
GET ID: Sevilla: 2833
GET ID: Villarreal: 2819
GET ID: Real Sociedad: 2824
GET ID: Real Betis: 2816
GET ID: Athletic Club: 2825
GET ID: Celta Vigo: 2821
GET ID: Almeria: 2858
GET ID: Getafe: 2859
GET ID: Mallorca: 2826
GET ID: Girona: 24264
GET ID: Granada: 33779
GET ID: Cadiz: 4488
GET ID: Alavés: 2885
GET ID: Rayo Vallecano: 2818
GET ID: Osasuna: 2820
GET ID: Las Palmas: 6577


### Obtenemos ultimos 29 partidos de todos los equipos de LaLiga

In [3]:
url = "https://sofascore.p.rapidapi.com/teams/get-last-matches"
equipos_data = {}

for team_id in ids_equipos:
    querystring = {"teamId": str(team_id), "pageIndex": "0"}

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()

        # Almacena la respuesta en el diccionario usando el ID del equipo como clave
        equipos_data[str(team_id)] = data
        print(f"GET info de {team_id}")
    else:
        print(f"Error al obtener datos para el equipo con ID {team_id}. Código de estado: {response.status_code}")

GET info de 2829
GET info de 2817
GET info de 77879
GET info de 2828
GET info de 2833
GET info de 2819
GET info de 2824
GET info de 2816
GET info de 2825
GET info de 2821
GET info de 2858
GET info de 2859
GET info de 2826
GET info de 24264
GET info de 33779
GET info de 4488
GET info de 2885
GET info de 2818
GET info de 2820
GET info de 6577


### Selecionamos el id de los últimos partidos disputados de todos los equipos de LaLiga

In [47]:
ids_matches = []

for equipo_id in ids_equipos:
    for i in range(2):
        match_id = equipos_data[str(equipo_id)]["events"][25+i]["id"]
        ids_matches.append(match_id)

# Print all the stored match IDs
print("Matches IDs:")
print("Total IDs:" + str(len(ids_matches)))
print(ids_matches)


Matches IDs:
Total IDs:40
[11605847, 11369354, 11369353, 11369359, 11695187, 11478200, 11369340, 11369350, 11369364, 11605843, 11369356, 11369366, 11369363, 11605831, 11369361, 11608617, 11369344, 11369352, 11369349, 11369360, 11369352, 11369358, 11369341, 11369349, 11369350, 11369363, 11369348, 11369358, 11369335, 11369353, 11369336, 11369348, 11369351, 11369366, 11369346, 11369355, 11369354, 11733377, 11369356, 11369357]


### Obtenemos los incidentes de todos de todos los últimos partidos disputados de todos los partidos de LaLiga

In [48]:
url = "https://sofascore.p.rapidapi.com/matches/get-incidents"

resultados = {}  # Diccionario para almacenar los resultados

for match_id in ids_matches:
    querystring = {"matchId": str(match_id)}
    response = requests.get(url, headers=headers, params=querystring)
    resultados[str(match_id)] = response.json()

#### Archivo con todas las tarjetas rojas de todos los últimos partidos disputados de todos los partidos de LaLiga
#### Archivo con todas las tarjetas amarillas de todos los últimos partidos disputados de todos los partidos de LaLiga
#### Archivo con todas las sustituciones de todos los últimos partidos disputados de todos los partidos de LaLiga
#### Archivo con todas los goles de todos los últimos partidos disputados de todos los partidos de LaLiga
#### Archivo con todas los asistentes de todos los últimos partidos disputados de todos los partidos de LaLiga

In [49]:
tarjetasAmarillas = [] 
tarjetasRojas = [] 
sustituciones = [] 
goleadores= [] 
asistentes= [] 

for match_id in ids_matches:
    incidencias=resultados[str(match_id)]['incidents']
    cantidad_incidencias = len(incidencias)
    
    for incidencia in range(cantidad_incidencias):
        tipo_incidencia = resultados[str(match_id)]['incidents'][incidencia]['incidentType']
        
        if tipo_incidencia == "card":
            tipo_card = resultados[str(match_id)]['incidents'][incidencia]['incidentClass']

            if tipo_card == "yellow":
                tarjetasAmarillas.append(resultados[str(match_id)]['incidents'][incidencia])

            elif tipo_card == "red":
                tarjetasRojas.append(resultados[str(match_id)]['incidents'][incidencia])
                
            else:
                print("wtf")
        elif tipo_incidencia == "substitution":
            sustituciones.append(resultados[str(match_id)]['incidents'][incidencia])
    
        elif tipo_incidencia == "goal":
            gol=resultados[str(match_id)]['incidents'][incidencia]
            
            goleadores.append(gol)
            
            if 'assist1' in gol:
                asistentes.append(resultados[str(match_id)]['incidents'][incidencia]['assist1'])
    
    
tarjetas_amarillas_json = json.dumps(tarjetasAmarillas)
tarjetas_rojas_json = json.dumps(tarjetasRojas)
sustituciones_json = json.dumps(sustituciones) 
goleadores_json = json.dumps(goleadores)
asistentes_json = json.dumps(asistentes)

file_name = 'datos_partidos/tarjetas_amarillas_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    file.write(tarjetas_amarillas_json)
    print("Guardado tarjetas Amarillas")
    
file_name = 'datos_partidos/tarjetas_rojas_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    file.write(tarjetas_rojas_json)
    print("Guardado tarjetas Rojas")
    
file_name = 'datos_partidos/sustituciones_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    file.write(sustituciones_json)
    print("Guardado sustituciones")
    
file_name = 'datos_partidos/goleadores_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    file.write(goleadores_json)
    print("Guardado goleadores")

file_name = 'datos_partidos/asistentes_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    file.write(asistentes_json)
    print("Guardado asistentes")

wtf
wtf
wtf
Guardado tarjetas Amarillas
Guardado tarjetas Rojas
Guardado sustituciones
Guardado goleadores
Guardado asistentes


### Listado de todos los "men of the match" de todos los últimos partidos disputados de todos los partidos de LaLiga

In [50]:
menMatch = {}

url = "https://sofascore.p.rapidapi.com/matches/get-best-players"

for match_id in ids_matches:
    querystring = {"matchId": str(match_id)}
    response = requests.get(url, headers=headers, params=querystring)
    menMatch[str(match_id)] = response.json()
    
# Guarda el diccionario en un archivo JSON
file_name = 'datos_partidos/men_of_the_match_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    json.dump(menMatch, file, indent=4)

print(f"Todos los datos guardados en{file_name}")

Todos los datos guardados endatos_partidos/men_of_the_match_ultimos_partidos_todos_equipos.json


### Listado de todos los jugadores que son titulares de todos los últimos partidos disputados de todos los partidos de LaLiga

In [51]:
url = "https://sofascore.p.rapidapi.com/matches/get-lineups"

titulares = {}

for match_id in ids_matches:
    querystring = {"matchId": str(match_id)}
    response = requests.get(url, headers=headers, params=querystring)
    titulares[str(match_id)] = response.json()
    
# Guarda el diccionario en un archivo JSON
file_name = 'datos_partidos/jugadores_titulares_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    json.dump(titulares, file, indent=4)

print(f"Todos los datos guardados en{file_name}")

Todos los datos guardados endatos_partidos/jugadores_titulares_ultimos_partidos_todos_equipos.json


### Listado de todas las estadisticas de todos los últimos partidos disputados de todos los partidos de LaLiga

In [ ]:
url = "https://sofascore.p.rapidapi.com/matches/get-statistics"

estadisticaspartidos = {}

for match_id in ids_matches:
    querystring = {"matchId": str(match_id)}
    response = requests.get(url, headers=headers, params=querystring)
    estadisticaspartidos[str(match_id)] = response.json()
    
# Guarda el diccionario en un archivo JSON
file_name = 'datos_partidos/estadisticas_ultimos_partidos_todos_equipos.json'
with open(file_name, 'w') as file:
    json.dump(estadisticaspartidos, file, indent=4)

print(f"Todos los datos guardados en{file_name}")